In [ ]:
import tensorflow as tf
from tensorflow import keras

In [ ]:
keras.__version__

In [ ]:
fmnist_data = keras.datasets.fashion_mnist

In [ ]:
(x_train, y_train), (x_test, y_test) = fmnist_data.load_data()